### Data Cleaning

In [1]:
#!pip install pygeohash
import pandas as pd
import numpy as np
import re
import pygeohash

In [2]:
# read in the csv files, merge the dfs
df1 = pd.read_csv("../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv")
df2 = pd.read_csv("../data/Resale flat prices based on registration date from Jan-2017 onwards.csv")
df1['date'] = pd.to_datetime(df1['month'])
df2['date'] = pd.to_datetime(df2['month'])

def parse_lease(lease_str):
    pattern = r"(\d+)\s*years(?:\s+(\d+)\s*months?)?"
    match = re.search(pattern, lease_str.lower())
    if match:
        years = int(match.group(1))
        return years
    return None

df2['remaining_lease'] = df2['remaining_lease'].apply(parse_lease)
df = pd.concat([df1,df2])

# converting using inflation rate to get adjusted prices
inflation_dict = {
    2012: 4.6,
    2013: 2.4,
    2014: 1.0,
    2015: -0.5,
    2016: -0.5,
    2017: 0.6,
    2018: 0.4,
    2019: 0.6,
    2020: -0.2,
    2021: 2.3,
    2022: 6.1,
    2023: 4.0,   
    2024: 2.5,   
    2025: 2.0,   # forecast
}

def compound_to_2025(original_price, sale_year, inflation_data):
    factor = 1.0
    for y in range(sale_year + 1, 2026):
        if y in inflation_data:
            factor *= (1 + inflation_data[y] / 100.0)
        else:
            factor *= 1.0
    return original_price * factor

df['adjusted_resale_price'] = df.apply(lambda row: compound_to_2025(row['resale_price'], row['date'].year, inflation_dict), axis=1)
df['adj_resale_price_per_sqm'] = df['adjusted_resale_price']/df['floor_area_sqm']
df = df.drop(['resale_price'], axis=1)

In [3]:
df["address"] = df["block"] + " " + df["street_name"]
unique_addresses = df["address"].unique()
unique_hdb_df = pd.DataFrame(unique_addresses, columns=["address"])

In [ ]:
# geocode lat long postal code for unique hdb blocks found in transactions
# call geocode_update to geocode for addresses, use OneMapa api token
from geocode_update import geocode_address
results = []
for add in unique_hdb_df['address']:
    codes = geocode_address(add)
    results.append(codes)

hdb_geocoded = pd.DataFrame(results)

#hdb_geocoded.to_csv("hdb_codes.csv", index=False)

In [4]:
# GETTING DISTANCES
import geohashing_and_distances as gd
hdb_geocoded = pd.read_csv("../data/hdb_codes.csv")

hdb_geocoded = gd.dist_good_school(hdb_geocoded)
hdb_geocoded = gd.dist_to_cbd(hdb_geocoded)

hdb_geocoded['geohash'] =\
(hdb_geocoded.apply(lambda row: pygeohash.encode(row['latitude'],row['longitude']),axis=1))
df = pd.merge(df, hdb_geocoded, on = 'address', how='left')

# function call for getting mrt 
final_df = gd.dist_nearest_mrt(df)
final_df.isna().sum()   # check for NA

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
remaining_lease             0
date                        0
adjusted_resale_price       0
adj_resale_price_per_sqm    0
address                     0
latitude                    0
longitude                   0
postal_code                 0
min_dist_sch                0
min_dist_cbd                0
geohash                     0
min_dist_mrt                0
dtype: int64

In [ ]:
# handling the categorical variables for storey range and for the different flat types found 
def calculate_storey_median(storey_range):
    storey_min, storey_max = map(int, storey_range.split(' TO '))
    return (storey_min + storey_max) / 2

final_df['storey_median'] = final_df['storey_range'].apply(calculate_storey_median)
final_df = pd.get_dummies(final_df, columns=['flat_type'])
final_df = final_df.drop(['date'], axis=1)

In [14]:
# exporting final_df, this is the transaction datasest 
final_df.to_csv("hdb_final_dataset.csv", index=False)